In [160]:
!pip install --upgrade pip
!pip install -IV gremlinpython==3.4.1

     |████████████████████████████████| 1.5 MB 26.9 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.1.2
    Uninstalling pip-21.1.2:
      Successfully uninstalled pip-21.1.2
  Using cached gremlinpython-3.4.1-py2.py3-none-any.whl (52 kB)
  Using cached tornado-4.5.3-cp36-cp36m-linux_x86_64.whl
  Using cached isodate-0.6.0-py2.py3-none-any.whl (45 kB)
  Using cached aenum-3.1.0-py3-none-any.whl (123 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.
notebook 6.2.0 requires tornado>=6.1, but you have tornado 4.5.3 which is incompatible.
jupyterlab 3.0.9 requires tornado>=6.1.0, but you have tornado 4.5.3 which is incompatible.
jupyter-server 1.4.1 requires tornado>=6.1.0, but you have tornado 4.5.3 whic

In [6]:
!pip install imdbpy

     |████████████████████████████████| 298 kB 36.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python3.6 -m pip install --upgrade pip' command.


In [1]:
# import packages
import pandas as pd
import imdb
import requests
from lxml import etree
import urllib.request
from io import BytesIO
from gremlin_python.driver import client

from gremlin_python.process.traversal import P

from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Cardinality

from gremlin_python import statics
from gremlin_python.process.anonymous_traversal import traversal
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection
from gremlin_python.process.traversal import T
from gremlin_python.process.traversal import Order
from gremlin_python.process.traversal import Cardinality
from gremlin_python.process.traversal import Column
from gremlin_python.process.traversal import Direction
from gremlin_python.process.traversal import Operator
from gremlin_python.process.traversal import P
from gremlin_python.process.traversal import Pop
from gremlin_python.process.traversal import Scope
from gremlin_python.process.traversal import Barrier
from gremlin_python.process.traversal import Bindings
from gremlin_python.process.traversal import WithOptions
from gremlin_python.process.traversal import TextP

In [2]:
# define function to get the entire imdb dataset
def get_all_movies():
    session = requests.Session()
    url = "https://datasets.imdbws.com/"
    r = session.get(url)
    tree = etree.HTML(r.content)
    fd = urllib.request.urlopen(tree.xpath('/html/body/ul[3]/a')[0].attrib['href'])
    df_movies = pd.read_csv(BytesIO(fd.read()), compression='gzip', sep='\t')
    return df_movies

In [3]:
# define function to get a subset of the imdb dataset
def get_some_movies(rows):
    session = requests.Session()
    url = "https://datasets.imdbws.com/"
    r = session.get(url)
    tree = etree.HTML(r.content)
    fd = urllib.request.urlopen(tree.xpath('/html/body/ul[3]/a')[0].attrib['href'])
    df_movies = pd.read_csv(BytesIO(fd.read()), compression='gzip', sep='\t')
    return df_movies.iloc[:rows, :]

In [4]:
# define function to get data from a sample file
def get_movies_from_file(url):
    fd = urllib.request.urlopen(url)
    df_movies = pd.read_csv(BytesIO(fd.read()))
    return df_movies

In [5]:
# define query to get graph traversal object
def get_graph(neptune_url):
    try:
        g = traversal().withRemote(DriverRemoteConnection('wss://%s:8182/gremlin' % neptune_url, 'g'))
        return g
    except Exception as e:
        raise Exception('Could not connect to Neptune')

In [6]:
# define function to run queries in Python
def run_query(query, neptune_url):
    graph = None
    try:
        graph = client.Client(neptune_url, 'g')
        result_set = graph.submit(query)
        future_results = result_set.all()
        results = future_results.result()
        return results
    except Exception as ex:
        raise Exception(f'Failed to connect to graph database, error={str(ex)} ...')
    finally:
        if graph:
            graph.close()

In [181]:
# define function to delete existing graph
def delete_graph():
    g.V().drop().iterate()

In [193]:
# define function to load data into database
# one of the functions given above can be used to create a dataframe
def load_data(df_movies):
    # iterate through the data and  create graph
    for index, movie_row in df_movies.iterrows():
        try:
            # add movie node
            movie = ia.get_movie(movie_row['tconst'].replace('tt', ''))
            movie_node = g.addV('Movie').property('title', movie['title']).next()
            print(movie_node)
            try:
                g.V(movie_node).property('year', movie['year']).next()
            except:
                pass
            try:
                g.V(movie_node).property('votes', movie['votes']).next()
            except:
                pass
            try:
                g.V(movie_node).property('rating', movie['rating']).next()
            except:
                pass
            try:
                g.V(movie_node).property('akas', movie['akas']).next()
            except:
                pass

            # add actor nodes
            try:
                actors = movie['cast']
                if actors:
                    for actor in actors:
                        # check if actor is already available
                        if g.V().has('Person', 'name', actor['name']).valueMap().toList():
                            actor_node = g.V().has('Person', 'name', actor['name']).next()

                        # create new actor if not available
                        else:
                            actor_node = g.addV('Person').property('name', actor['name']).next()

                        # create edge between movie and actor
                        g.V(actor_node).addE('ACTED_IN').to(movie_node).iterate()

            except:
                pass

            # add director nodes
            try:
                directors = movie['directors']
                if directors:
                    for director in directors:
                        # check if director is already available
                        if g.V().has('Person', 'name', director['name']).valueMap().toList():
                            director_node = g.V().has('Person', 'name', director['name']).next()

                        # create new director if not available
                        else:
                            director_node = g.addV('Person').property('name', director['name']).next()

                        g.V(director_node).addE('DIRECTED').to(movie_node).iterate()
            except:
                pass

            # add producer nodes
            try:
                producers = movie['producers']
                if producers:
                    for producer in producers:
                        # check if producer is already available
                        if g.V().has('Person', 'name', producer['name']).valueMap().toList():
                            producer_node = g.V().has('Person', 'name', producer['name']).next()

                        # create new producer if not available
                        else:
                            producer_node = g.addV('Person').property('name', producer['name']).next()

                        g.V(producer_node).addE('PRODUCED').to(movie_node).iterate()
            except:
                pass

            # add genre nodes
            try:
                genres = movie['genres']
                if genres:
                    for genre in genres:
                        # check if genre is already available
                        if g.V().has('Genre', 'type', genre).valueMap().toList():
                            genre_node = g.V().has('Genre', 'type', genre).next()

                        # create genre if not available
                        else:
                            genre_node = g.addV('Genre').property('type', genre).next()

                        g.V(movie_node).addE('IS').to(genre_node).iterate()
            except:
                pass

            # add production company nodes
            try:
                production_companies = movie['production companies']
                if production_companies:
                    for production_company in production_companies:
                        # check if company is already available
                        if g.V().has('Company', 'name', production_company['name']).valueMap().toList():
                            production_company_node = g.V().has('Company', 'name', production_company['name']).next()

                        # create company if not available
                        else:
                            production_company_node = g.addV('Company').property('name', production_company['name']).next()

                        g.V(production_company_node).addE('PRODUCED').to(movie_node).iterate()

            except:
                pass

            # add distribution company nodes
            try:
                distributors = movie['distributors']
                if distributors:
                    for distributor in distributors:
                        # check if distributor is already available
                        if g.V().has('Company', 'name', distributor['name']).valueMap().toList():
                            distributor_node = g.V().has('Company', 'name', distributor['name']).next()

                        # create distributor if not available
                        else:
                            distributor_node = g.addV('Company').property('name', distributor['name']).next()

                        # create edge between distributor and movie
                        g.V(distributor_node).addE('DISTRIBUTED').to(movie_node).iterate()
            except:
                pass
            
            # add country nodes
            try:
                countries = movie['countries']
                if countries:
                    for country in countries:
                        # check if country is already available
                        if g.V().has('Country', 'name', country).valueMap().toList():
                            country_node = g.V().has('Country', 'name', country).next()
                            
                        # create country if not available
                        else:
                            country_node = g.addV('Country').property('name', country).next()
                            
                        # create edge between movie and country
                        g.V(movie_node).addE('BASED_IN').to(country_node).iterate()
            except:
                pass
        except Exception as e:
            print(e)

In [161]:
# get movies from the sample file
df_movies = get_movies_from_file('https://github.com/MinuraPunchihewa/IMDB-Data-Gremlin/blob/main/imdb-data-sample-1000.csv?raw=true')

In [8]:
# instantiate imdbpy object
ia = imdb.IMDb()

In [162]:
# explore movie data
df_movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [186]:
# get graph traversal object
g = get_graph()

In [205]:
# delete existing graph
delete_graph()

In [208]:
# load data
load_data(df_movies)

v[96bd4070-ab7f-eb28-e98a-0fd3ea22a41b]
v[f0bd4070-aefc-d533-bc25-06e5a8779bd1]
v[16bd4070-b22d-f651-a26a-9c43f2921bb6]
v[66bd4070-b5b0-6984-6c67-482fe5c15295]
v[d2bd4070-b9bb-239f-bb83-e56e486a2c90]
v[98bd4070-bedc-0f9f-a08d-b7d0d7bbf829]
v[3ebd4070-c24e-f5d8-dd70-5afa35ad36ae]
v[56bd4070-c5bf-4fa1-c640-1ae9558ee2dd]
v[20bd4070-c868-0ff9-14b1-6f2d188e8a57]
v[7ebd4070-cde4-f196-2288-6c7d22702d4a]
v[3ebd4070-d30e-e146-a6f4-f2eb515ffcd6]
v[9ebd4070-d6c9-72ec-05e3-2446d513007f]
v[b4bd4070-daaa-4cee-4017-6260ed46629a]
v[42bd4070-de16-5704-bd30-b12fe8947866]
v[c6bd4070-e1da-520b-5485-9037d85d7557]
v[b2bd4070-e4c6-1e77-b9ec-afd25db23781]
v[9abd4070-e874-daad-2745-f7f3466c845a]
v[88bd4070-ed47-3232-715f-7e0f4e129255]
v[c8bd4070-eff7-0776-d08d-de8e48c6bf6c]
v[eebd4070-f335-aee6-442e-85b18766879c]
v[40bd4070-f6a8-1c6c-247a-939f65840c4f]
v[cebd4070-f9b6-2420-f6d6-073bbfaac4e7]
v[babd4070-fcc5-c54b-a9ce-b27324c5b874]
v[3ebd4070-ffb6-70d0-a0c5-cb4fad6962e6]
v[52bd4071-02f3-c34e-8467-a22ad6717dcd]
